In [1]:
import psycopg2 as pg2
import pandas as pd
from sqlalchemy import create_engine


con = pg2.connect(host='this_postgres',
                  user='postgres',
                  password='password',
                  database='postgres')
con.autocommit = True
cur = con.cursor()

def select(sql):
    return pd.read_sql(sql,con)

In [2]:
def get_investor_apartment_affordability():
    sql = '''
        with temp as (select suburb,
                             sub_ref,
                             sold_price,
                             date_trunc
                                 (
                                     'MONTH',
                                     sold_date
                                 )::date                                             as sold_month,
                             1                                                       as quantity_sold,
                             case when sold_price <= 800000 * 1.05 then 1 else 0 end as affordability_counter


                      from sold_history_all_subs
                      where sold_date >= '2016-01-01'
                        and property_type = 'Apartment / Unit / Flat'
                        and bedrooms = '2'
                        and sold_date >= date_trunc('month', current_date) - INTERVAL '5 months')

        select suburb, sub_ref, round(avg(sold_price),0) as average_price,sum(quantity_sold) as total_sold, sum(affordability_counter) as total_affordable
        , round(sum(affordability_counter)*100.0/sum(quantity_sold),1) as affordability_percentage
        from temp
        group by 1, 2
        order by 6 desc,4 desc
        '''
    df_invest_apartment = select(sql)
    return df_invest_apartment

In [3]:
def get_invest_distance_to_cbd():
    sql = '''
        select invest_affordable_apartments.suburb,
               invest_affordable_apartments.sub_ref,
               invest_affordable_apartments.average_price,
               distance_to_cbd from invest_affordable_apartments
        left join syd_subs_lat_lng
        on invest_affordable_apartments.sub_ref = syd_subs_lat_lng.sub_ref
        order by 4
        '''
    df_invest_distance_to_cbd = select(sql)
    return df_invest_distance_to_cbd

In [4]:
def get_invest_apartment_biggest_loss():
    sql = '''
        select buy_month,
               sell_month,
               duration,
               suburb,
               -profit                                         as loss,
               round(-profit * 100.0 / buy_month_avg_price, 1) as loss_margin,
               buy_month_avg_price,
               sell_month_avg_price,
               buy_month_total_sold,
               sell_month_total_sold
        from invest_affordable_apartments_max_min_price_joined
        where profit = min_profit
        order by loss_margin
        '''
    df_invest_apartment_biggest_loss = select(sql)
    return df_invest_apartment_biggest_loss

In [ ]:
def get_invest_apartment_recovery():
    sql = '''
        with temp as (select invest_affordable_apartments_loss_acceptable.suburb,
               buy_month,
               sell_month,
               sold_month                                         as recovery_month,
               duration,
               (DATE_PART('year', invest_affordable_apartments_monthly_sold_history.sold_month) - DATE_PART('year', invest_affordable_apartments_loss_acceptable.sell_month)) * 12 +
               (DATE_PART('month', invest_affordable_apartments_monthly_sold_history.sold_month) -
                DATE_PART('month', invest_affordable_apartments_loss_acceptable.sell_month)) as recovery_duration,
               loss,
               average_price - buy_month_avg_price as recovery_profit_or_loss,
               max(average_price - buy_month_avg_price) over (partition by invest_affordable_apartments_loss_acceptable.suburb) as biggest_recovery_profit_or_loss,
               loss_margin,
               buy_month_avg_price,
               sell_month_avg_price,
               average_price                                      as recovery_price
        from invest_affordable_apartments_loss_acceptable
                 left join invest_affordable_apartments_monthly_sold_history
                           on invest_affordable_apartments_monthly_sold_history.sold_month >
                              invest_affordable_apartments_loss_acceptable.sell_month
                               and invest_affordable_apartments_loss_acceptable.suburb =
                                   invest_affordable_apartments_monthly_sold_history.suburb)
        select suburb,
               buy_month,
               sell_month,
               recovery_month,
               duration,
               recovery_duration,
               loss,
               recovery_profit_or_loss,
               round(cast(recovery_profit_or_loss * 100.0 / buy_month_avg_price as numeric), 1) as recovery_margin,
               loss_margin,
               buy_month_avg_price,
               sell_month_avg_price,
               recovery_price
        from temp
        where recovery_profit_or_loss = biggest_recovery_profit_or_loss
        order by recovery_margin desc
        '''
    df_invest_apartment_recovery = select(sql)
    return df_invest_apartment_recovery

In [ ]:
def get_invest_house_affordability():
    sql = '''
        with temp as (select suburb,
                             sub_ref,
                             sold_price,
                             date_trunc
                                 (
                                     'MONTH',
                                     sold_date
                                 )::date                                             as sold_month,
                             1                                                       as quantity_sold,
                             case when sold_price <= 2000000 * 1.05 then 1 else 0 end as affordability_counter


                      from sold_history_all_subs
                      where sold_date >= '2016-01-01'
                        and property_type = 'House'
                        and bedrooms = '4'
                        and sold_date >= date_trunc('month', current_date) - INTERVAL '5 months')

        select suburb, sub_ref, round(avg(sold_price),0) as average_price,sum(quantity_sold) as total_sold, sum(affordability_counter) as total_affordable
        , round(sum(affordability_counter)*100.0/sum(quantity_sold),1) as affordability_percentage
        from temp
        group by 1, 2
        order by 6 desc,4 desc
        '''
    df_invest_house = select(sql)
    df_invest_house_affordability = df_invest_house.loc[(df_invest_house['affordability_percentage'] >= 30) & (df_invest_house['total_sold'] >= 18)]

    return df_invest_house_affordability

In [ ]:
def get_invest_house_cbd_35km():
    sql = '''
        select invest_affordable_houses.suburb,
               invest_affordable_houses.sub_ref,
               invest_affordable_houses.average_price,
               distance_to_cbd from invest_affordable_houses
        left join syd_subs_lat_lng
        on invest_affordable_houses.sub_ref = syd_subs_lat_lng.sub_ref
        order by 4
            '''
    df_invest_distance_to_cbd = select(sql)
    df_invest_house_cbd_35km = df_invest_distance_to_cbd.loc[(df_invest_distance_to_cbd['distance_to_cbd'] <= 35)]
    return df_invest_house_cbd_35km

In [ ]:
def get_invest_house_loss_acceptable():
    sql = '''
        select buy_month,
               sell_month,
               duration,
               suburb,
               -profit                                         as loss,
               round(-profit * 100.0 / buy_month_avg_price, 1) as loss_margin,
               buy_month_avg_price,
               sell_month_avg_price,
               buy_month_total_sold,
               sell_month_total_sold
        from invest_affordable_houses_max_min_price_joined
        where profit = min_profit
        order by loss_margin
            '''
    df_invest_house_biggest_loss = select(sql)
    df_invest_house_loss_acceptable = df_invest_house_biggest_loss.loc[(df_invest_house_biggest_loss['duration'] >= 9) & (df_invest_house_biggest_loss['loss_margin'] <= 40)]
    return df_invest_house_loss_acceptable

In [ ]:
def get_invest_house_recovery():
    sql = '''
        with temp as (select invest_affordable_houses_loss_acceptable.suburb,
               buy_month,
               sell_month,
               sold_month                                         as recovery_month,
               duration,
               (DATE_PART('year', invest_affordable_houses_monthly_sold_history.sold_month) - DATE_PART('year', invest_affordable_houses_loss_acceptable.sell_month)) * 12 +
               (DATE_PART('month', invest_affordable_houses_monthly_sold_history.sold_month) -
                DATE_PART('month', invest_affordable_houses_loss_acceptable.sell_month)) as recovery_duration,
               loss,
               average_price - buy_month_avg_price as recovery_profit_or_loss,
               max(average_price - buy_month_avg_price) over (partition by invest_affordable_houses_loss_acceptable.suburb) as biggest_recovery_profit_or_loss,
               loss_margin,
               buy_month_avg_price,
               sell_month_avg_price,
               average_price                                      as recovery_price
        from invest_affordable_houses_loss_acceptable
                 left join invest_affordable_houses_monthly_sold_history
                           on invest_affordable_houses_monthly_sold_history.sold_month >
                              invest_affordable_houses_loss_acceptable.sell_month
                               and invest_affordable_houses_loss_acceptable.suburb =
                                   invest_affordable_houses_monthly_sold_history.suburb)
        select suburb,
               buy_month,
               sell_month,
               recovery_month,
               duration,
               recovery_duration,
               loss,
               recovery_profit_or_loss,
               round(cast(recovery_profit_or_loss * 100.0 / buy_month_avg_price as numeric), 1) as recovery_margin,
               loss_margin,
               buy_month_avg_price,
               sell_month_avg_price,
               recovery_price
        from temp
        where recovery_profit_or_loss = biggest_recovery_profit_or_loss
        order by recovery_margin desc
            '''
    df_invest_house_recovery = select(sql)
    return df_invest_house_recovery

In [ ]:
def get_home_buyer_house_affordable():
    sql = '''
        with raw_data as (select suburb,
                                     sub_ref,
                                     sold_date,
                                     date_trunc('MONTH', sold_date)::date                     as sold_month,
                                     sold_price,
                                     bedrooms,
                                     bathrooms,
                                     1                                                        as property_counter,
                                     case when sold_price <= 2000000 * 1.05 then 1 else 0 end as affordability_counter
                              from sold_history_all_subs
                              where sold_date >= date_trunc('month', current_date) - INTERVAL '5 months'
                                and property_type = 'House'
                                and sold_price <= 6000000
                                and bedrooms in (3, 4))
               , affordability as (select suburb,
                                          sub_ref,
                                          round(avg(sold_price), 0)                                            as avg_price,
                                          round(sum(affordability_counter) * 100.0 / sum(property_counter), 0) as affordability_percentage,
                                          sum(property_counter)                                                as total_properties_sold,
                                          sum(affordability_counter)                                           as total_properies_affordable

                                   from raw_data
                                   group by 1, 2
                                   having sum(affordability_counter) * 100.0 / sum(property_counter) >= 50
                                   order by avg_price desc)
            select *
            from affordability
            '''
    df_home_buyer_house_affordable = select(sql)
    return df_home_buyer_house_affordable

In [ ]:
def get_home_buyer_house_region_selected():
    sql = '''
    select home_buyer_house_affordability_data.suburb,
           home_buyer_house_affordability_data.sub_ref,
           avg_price,
           affordability_percentage,
           total_properties_sold,
           total_properies_affordable,
           local_goverment_area,
           distance_to_cbd
    from home_buyer_house_affordability_data
             left join syd_subs_lat_lng
                       on home_buyer_house_affordability_data.sub_ref = syd_subs_lat_lng.sub_ref
    where local_goverment_area like 'Inner West%'
       or local_goverment_area like 'Ryde%'
       or local_goverment_area like 'Northern Beaches%'
        and distance_to_cbd <= 25'''
    df_home_buyer_house_region_selected = select(sql)
    return df_home_buyer_house_region_selected

In [ ]:
def get_rozelle_recent_sale():
    sql = '''
        select sold_month, suburb, total_houses, round(average_price, 0) as avg_price
        from home_buyer_houses_stats
        where suburb like 'ROZELLE%'
          and sold_month >= date_trunc('month', current_date) - INTERVAL '4 months'
          '''
    df_rozelle_recent_sale = select(sql)
    return df_rozelle_recent_sale

In [ ]:
def get_rozelle_sp():
    sql = '''
        select "Suburb",
               bedrooms,
               avg_days_on_market
        from suburb_profile
        where "Suburb" = 'rozelle'
          and type = 'House'
          and bedrooms in (3, 4)
          '''
    df_rozelle_sp = select(sql)
    return df_rozelle_sp

In [ ]:
def get_rozelle_rolling_price():
    sql = '''
        with raw_data as (select suburb,
                                 sub_ref,
                                 date_trunc('MONTH', sold_date)::date as sold_month,
                                 sold_type,
                                 property_type,
                                 sold_price,
                                 sold_price_desc,
                                 bedrooms,
                                 bathrooms,
                                 parkings,
                                 landsize_m²,
                                 url
                          from sold_history_all_subs
                          where 
                            property_type = 'House'
                            and sold_price <= 6000000
                            and bedrooms in (3, 4)
                          order by suburb, sold_month)
           , monthly_data as (select suburb,
                                     sub_ref,
                                     sold_month,
                                     round(avg(sold_price), 0) as avg_monthly_sold_price,
                                     count(1)                  as total_sold
                              from raw_data
                              group by suburb, sub_ref, sold_month
                              having count(1) >= 3
                              order by 1, 3)
           , rolling as (select *,
                                round(avg(avg_monthly_sold_price)
                                      over (partition by suburb order by sold_month rows between 3 preceding and 1 preceding ),
                                      0) as last_3_month_average_sold_price
                         from monthly_data)
        select suburb,
               sold_month,
               avg_monthly_sold_price,
               last_3_month_average_sold_price,
               round((avg_monthly_sold_price - last_3_month_average_sold_price) * 100.0 / last_3_month_average_sold_price,
                     2) as percentage_change,
               total_sold
        from rolling
        where last_3_month_average_sold_price is not null
          and suburb like 'ROZELLE NSW%'
          and sold_month >= '2020-01-01'
          '''
    df_rozelle_rolling_price = select(sql)
    return df_rozelle_rolling_price

In [ ]:
def get_rozelle_surrounding_subs():
    sql = '''
        select selected_suburb,
               surrounding_suburbs,
               distance,
               avg_price,
               affordability_percentage,
               total_properties_sold,
               total_properies_affordable
        from surrounding_suburbs
                 left join home_buyer_house_affordability_data
                           on surrounding_suburbs.surrounding_suburb_ref = home_buyer_house_affordability_data.sub_ref
        where selected_suburb = 'Rozelle'
          and distance > 0
          and affordability_percentage is not null
        order by distance
        limit 10 
          '''
    df_rozelle_surrounding_subs = select(sql)
    return df_rozelle_surrounding_subs

In [ ]:
def get_home_buyer_apartment_affordable():
    sql = '''
    with raw_data as (select suburb,
                                 sub_ref,
                                 sold_date,
                                 date_trunc('MONTH', sold_date)::date                     as sold_month,
                                 sold_price,
                                 bedrooms,
                                 bathrooms,
                                 1                                                        as property_counter,
                                 case when sold_price <= 800000 * 1.05 then 1 else 0 end as affordability_counter
                          from sold_history_all_subs
                          where sold_date >= date_trunc('month', current_date) - INTERVAL '5 months'
                            and property_type = 'Apartment / Unit / Flat'
                            and sold_price <= 6000000
                            and sold_price > 10000
                            and bedrooms = 2)
           , affordability as (select suburb,
                                      sub_ref,
                                      round(avg(sold_price), 0)                                            as avg_price,
                                      round(sum(affordability_counter) * 100.0 / sum(property_counter), 0) as affordability_percentage,
                                      sum(property_counter)                                                as total_properties_sold,
                                      sum(affordability_counter)                                           as total_properies_affordable

                               from raw_data
                               group by 1, 2
                               having sum(affordability_counter) * 100.0 / sum(property_counter) >= 50
                               order by avg_price desc)
        select *
        from affordability
        '''
    df_home_buyer_apartment_affordable = select(sql)
    return df_home_buyer_apartment_affordable

In [ ]:
def get_home_buyer_apartment_affordable():
    sql = '''
    select home_buyer_apartment_affordability_data.suburb,
           home_buyer_apartment_affordability_data.sub_ref,
           avg_price,
           affordability_percentage,
           total_properties_sold,
           total_properies_affordable,
           local_goverment_area,
           distance_to_cbd
    from home_buyer_apartment_affordability_data
             left join syd_subs_lat_lng
                       on home_buyer_apartment_affordability_data.sub_ref = syd_subs_lat_lng.sub_ref
    where distance_to_cbd <= 10'''
    df_home_buyer_apartment_region_selected = select(sql)
    return df_home_buyer_apartment_region_selected

In [ ]:
def get_rosebery_recent_sale():
    sql = '''
    select sold_month, suburb, total_houses, round(average_price, 0) as avg_price
    from home_buyer_apartments_stats
    where suburb like 'ROSEBERY%'
      and sold_month >= date_trunc('month', current_date) - INTERVAL '4 months'
    '''
    df_rosebery_recent_sale = select(sql)
    return df_rosebery_recent_sale

In [ ]:
def get_rosebery_sp():
    sql = '''
        select "Suburb",
           bedrooms,
           avg_days_on_market
    from suburb_profile
    where "Suburb" = 'rosebery'
      and type = 'Unit'
      and bedrooms = 2
  '''
    df_rosebery_sp = select(sql)
    return df_rosebery_sp

In [ ]:
def get_rosebery_rolling_price():
    sql = '''
        with raw_data as (select suburb,
                                 sub_ref,
                                 date_trunc('MONTH', sold_date)::date as sold_month,
                                 sold_type,
                                 property_type,
                                 sold_price,
                                 sold_price_desc,
                                 bedrooms,
                                 bathrooms,
                                 parkings,
                                 landsize_m²,
                                 url
                          from sold_history_all_subs
                          where 
                            property_type = 'Apartment / Unit / Flat'
                            and sold_price <= 6000000
                            and bedrooms = 2
                          order by suburb, sold_month)
           , monthly_data as (select suburb,
                                     sub_ref,
                                     sold_month,
                                     round(avg(sold_price), 0) as avg_monthly_sold_price,
                                     count(1)                  as total_sold
                              from raw_data
                              group by suburb, sub_ref, sold_month
                              having count(1) >= 3
                              order by 1, 3)
           , rolling as (select *,
                                round(avg(avg_monthly_sold_price)
                                      over (partition by suburb order by sold_month rows between 3 preceding and 1 preceding ),
                                      0) as last_3_month_average_sold_price
                         from monthly_data)
        select suburb,
               sold_month,
               avg_monthly_sold_price,
               last_3_month_average_sold_price,
               round((avg_monthly_sold_price - last_3_month_average_sold_price) * 100.0 / last_3_month_average_sold_price,
                     2) as percentage_change,
               total_sold
        from rolling
        where last_3_month_average_sold_price is not null
          and suburb like 'ROSEBERY NSW%'
          and sold_month >= '2020-01-01' 
  '''
    df_rosebery_rolling_price = select(sql)
    return df_rosebery_rolling_price